In [1]:
import argparse
import math
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import albumentations as A
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from gan import Generator
from gan import Discriminator
from gan import weights_init
from dataset import HandDataset
import cv2
import timm
from ultralytics import YOLO
from PIL import Image
from gan import save_images, save_image
from gan import Trainer

/home/anton/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ds_path = 'archive/Hands/Hands'
img_size = 256
random_state = 69
device = 'cuda' if torch.cuda.is_available else 'cpu'
z_size = 512

df = pd.read_csv('archive/HandInfo.csv')
df

,id,age,gender,skinColor,accessories,nailPolish,aspectOfHand,imageName,irregularities
0,0,27,male,fair,0,0,dorsal right,Hand_0000002.jpg,0
1,0,27,male,fair,0,0,dorsal right,Hand_0000003.jpg,0
2,0,27,male,fair,0,0,dorsal right,Hand_0000004.jpg,0
3,0,27,male,fair,0,0,dorsal right,Hand_0000005.jpg,0
4,0,27,male,fair,0,0,dorsal right,Hand_0000006.jpg,0
...,...,...,...,...,...,...,...,...,...
11071,1589,22,female,fair,0,0,palmar left,Hand_0011740.jpg,0
11072,1589,22,female,fair,0,0,palmar left,Hand_0011741.jpg,0
11073,1589,22,female,fair,0,0,palmar left,Hand_0011742.jpg,0
11074,1589,22,female,fair,0,0,palmar left,Hand_0011743.jpg,0


In [3]:
workers = 4
batch_size = 32
image_size = 256
nc = 3
nz = 500
ngf = 64
ndf = 64
num_epochs = 300
lr = 0.0002
beta1 = 0.5
ngpu = 1

In [4]:
yolo = YOLO('yolo11n.pt')
yolo = yolo.to(device)

res = yolo.train(data='/home/anton/PycharmProjects/another_projects/bio/hand-palm-detection.v2i.yolov11/data.yaml', epochs=200)

results = yolo([os.path.join(ds_path, fn) for fn in os.listdir(ds_path)][:3])
results

New https://pypi.org/project/ultralytics/8.3.27 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/anton/PycharmProjects/another_projects/bio/hand-palm-detection.v2i.yolov11/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt

RuntimeError: Dataset '/home/anton/PycharmProjects/another_projects/bio/hand-palm-detection.v2i.yolov11/data.yaml' error ❌ '/home/anton/PycharmProjects/another_projects/bio/hand-palm-detection.v2i.yolov11/data.yaml' does not exist

In [5]:
yolo.val()

YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


FileNotFoundError: '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' does not exist

In [ ]:
for result in results:
    boxes = result.boxes
    print(boxes)

In [7]:
from pathlib import Path

path_to_model = '/home/anton/PycharmProjects/another_projects/bio/best.pt'
best_yolo = YOLO(path_to_model)

if not Path('cropped_images').exists():
    os.mkdir('cropped_images')
    
all_fns = [os.path.join(ds_path, fn) for fn in os.listdir(ds_path)]

for fn in tqdm(all_fns):
    result = best_yolo.predict(fn, show=False, verbose=False)
    boxes = result[0].boxes.xyxy
    if len(boxes) > 0:
        x1, y1, x2, y2 = list(map(int, boxes.detach().cpu().numpy()[0]))
    
        img_name = fn.split('/')[-1]
        img = cv2.imread(fn)
        
        x1, x2 = max(int(x1 - 0.2 * img.shape[1]), 0), min(int(x2 + 0.2 * img.shape[1]), img.shape[1] - 2)
        y1, y2 = max(int(y1 - 0.2 * img.shape[0]), 0), min(int(y2 + 0.2 * img.shape[0]), img.shape[0] - 2)
        new_image = img[y1:y2+1, x1:x2+1]
        cv2.imwrite(os.path.join('cropped_images', img_name), new_image)
        continue
    
    image = cv2.imread(fn)
    img_name = fn.split('/')[-1]
    cv2.imwrite(os.path.join('cropped_images', img_name), image)

100%|█████████████████████████████████████| 11076/11076 [04:53<00:00, 37.79it/s]


In [8]:
def seed(random_state):
    torch.manual_seed(random_state)
    torch.cuda.manual_seed(random_state)
    torch.backends.cudnn.benchmark = True
    
seed(random_state)

In [9]:
transforms = A.Compose([
    A.Resize(img_size, img_size),
#     A.VerticalFlip(p=0.6),
#     A.HorizontalFlip(p=0.6),
#     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.6),
#     A.Normalize(mean=0.5, std=0.5)
])

In [10]:
ds_path_ = 'cropped_images'
dataset = HandDataset(ds_path_, df, transform=transforms)
dataset[0].shape

torch.Size([3, 256, 256])

In [11]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=False)

In [12]:
from collections import OrderedDict
lr = 0.0002

load = True

generator = Generator(ngpu).to(device)
# discriminator = timm.create_model('resnet18', pretrained=False, num_classes=2).to(device)
discriminator = Discriminator(ngpu).to(device)
weights_init(generator)
weights_init(discriminator)

if load:
    generator.load_state_dict(torch.load('generator.pt'))
    discriminator.load_state_dict(torch.load('discriminator.pt'))

criterion = nn.CrossEntropyLoss()

params = {
    'generator': generator,
    'discriminator': discriminator,
    'train_dl': dataloader, 
    'criterion': criterion,
    'lr': lr,
    'device': device,
    'epochs': num_epochs,
    'generator_path': 'generator.pt',
    'discriminator_path': 'discriminator.pt'
}
    
trainer = Trainer(**params)

In [ ]:
trainer.train()

Starting Training Loop...
epoch: 1


100%|██████████████| 347/347 [02:46<00:00,  2.09it/s, loss_d=0.314, loss_g=1.55]


epoch: 2


 29%|███▋         | 100/347 [00:52<02:08,  1.92it/s, loss_d=0.0028, loss_g=10.6]

In [22]:
# generator = Generator(ngpu).to(device)
# generator.load_state_dict(torch.load('generator.pt'))
# generator.eval()
# model_for_sampling = timm.create_model('resnet18', pretrained=True, num_classes=2).to(device)
# optimizer = torch.optim.Adam(model_for_sampling.parameters(), lr=lr)

# for epoch in range(5):
#     print(f'epoch: {epoch + 1}')
#     with tqdm(dataloader, total=len(dataloader), position=0, leave=True) as pbar:
#         for i, data in enumerate(pbar):
#             model_for_sampling.zero_grad()

#             real_cpu = data.to(device)

#             b_size = real_cpu.size(0)
#             label = torch.LongTensor([1 for j in range(b_size)]).to(device)

#             output = model_for_sampling(real_cpu)

#             loss_d_real = criterion(output, label)

#             loss_d_real.backward()
#             D_x = output.mean().item()
#             noise = torch.randn(b_size, nz, 1, 1, device=device)
#             fake = generator(noise)
#             label.fill_(fake_label)
#             output = model_for_sampling(fake.detach())

#             loss_d_fake = criterion(output, label)
            
#             loss_d_fake.backward()
#             D_G_z1 = output.mean().item()
#             loss_d = loss_d_real + loss_d_fake

#             optimizer.step()

#             pbar.set_postfix(
#                 OrderedDict(loss_d=loss_d.item())
#             )
        
#         iters += 1
#     torch.save(model_for_sampling.state_dict(), 'sep.pt')

epoch: 1


100%|██████████████████████████| 347/347 [02:23<00:00,  2.42it/s, loss_d=0.0156]


epoch: 2


100%|█████████████████████████| 347/347 [02:28<00:00,  2.33it/s, loss_d=0.00407]


epoch: 3


100%|████████████████████████| 347/347 [02:29<00:00,  2.32it/s, loss_d=0.000308]


epoch: 4


100%|████████████████████████| 347/347 [02:30<00:00,  2.30it/s, loss_d=0.000457]


epoch: 5


100%|████████████████████████| 347/347 [02:30<00:00,  2.31it/s, loss_d=0.000145]


In [45]:
generator = Generator(ngpu).to(device)
generator.load_state_dict(torch.load('generator.pt'))

discriminator = Discriminator(ngpu).to(device)
discriminator.load_state_dict(torch.load('discriminator.pt'))
# discriminator = timm.create_model('resnet18', num_classes=2).to(device)
# discriminator.load_state_dict(torch.load('sep.pt'))

discriminator.eval()
generator.eval()

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(500, 2048, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(2048, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Ba

In [50]:
bs = 32
steps = 1000

if not Path('generated').exists():
    os.mkdir('generated')

    
cnt = 0
for step in tqdm(range(steps)):
    with torch.no_grad():
        noise = torch.randn(bs, nz, 1, 1, device=device)
        images = generator(noise)
        probs = discriminator(images)
        
        probs = probs.softmax(dim=1)
#         labels = probs.argmax(dim=1).detach().cpu().numpy()
        for i in range(bs):
            if probs[i, 1] >= 0.6:
                save_image(images[i, ...].permute(1, 2, 0).detach().cpu().numpy(), 
                           os.path.join('generated', str(cnt) + '.png'))
                cnt += 1

100%|███████████████████████████████████████| 1000/1000 [00:46<00:00, 21.72it/s]
